In [38]:
# Import the dependencies.
from pathlib import Path
from sqlalchemy import create_engine
import pandas as pd
import csv
import sqlite3

In [39]:
# Define path to CSV and table name
csv_file_path = "processed_data_cleaned.csv"
sqlite_db_name = "spotify_db.sqlite"
table_name = "spotify_song"

In [40]:
# Create a new SQLite database
conn = sqlite3.connect(sqlite_db_name)

In [41]:
# Create the table 
with open(csv_file_path, "r") as csvfile:
    reader = csv.reader(csvfile)
    # Get field names from the first row of CSV (with headers)
    field_names = next(reader)
    field_definitions = ", ".join([f"{field} TEXT" for field in field_names])
    create_table_query = f"""CREATE TABLE {table_name} ({field_definitions})"""
    conn.execute(create_table_query)

In [42]:
# Import data from CSV file (with headers)
with open(csv_file_path, "r") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row (assuming headers are present)
    insert_query = f"""INSERT INTO {table_name} ({", ".join(field_names)}) VALUES (?, ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"""
    conn.executemany(insert_query, reader)

In [43]:
# Commit changes and close connection
conn.commit()
conn.close()
print(f"SQLite database '{sqlite_db_name}' created successfully!")

SQLite database 'spotify_db.sqlite' created successfully!


In [44]:
# Create a reference to the file. 
database_path = Path("spotify_db.sqlite")

In [45]:
# Create Engine
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [46]:
# Query All Records in the the Database
spotify_data = pd.read_sql("SELECT * FROM spotify_song", conn)

In [47]:
spotify_data['stream_count']=spotify_data['stream_count'].astype(float)

In [48]:
spotify_data.dtypes

track_id              object
track_name            object
track_artist          object
track_popularity      object
playlist_genre        object
playlist_subgenre     object
danceability          object
energy                object
key                   object
loudness              object
mode                  object
speechiness           object
acousticness          object
instrumentalness      object
liveness              object
valence               object
tempo                 object
stream_count         float64
duration_sec          object
months                object
dtype: object

In [49]:
cols_to_convert = ['tempo', 'valence','liveness','instrumentalness','acousticness','speechiness','loudness','energy','danceability','duration_sec']

In [50]:
spotify_data[cols_to_convert] = spotify_data[cols_to_convert].astype(float)
print("Columns converted successfully!")

Columns converted successfully!


In [51]:
cols_to_int = ['track_popularity','key','mode','months']
spotify_data[cols_to_int] = spotify_data[cols_to_int].astype(int)
print("Columns converted successfully!")

Columns converted successfully!


In [52]:
spotify_data.dtypes

track_id              object
track_name            object
track_artist          object
track_popularity       int64
playlist_genre        object
playlist_subgenre     object
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
stream_count         float64
duration_sec         float64
months                 int64
dtype: object

In [53]:
spotify_data.head()

,track_id,track_name,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,stream_count,duration_sec,months
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,36892097.0,194.754,57
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,pop,dance pop,0.726,0.815,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,26063851.0,162.600,51
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,pop,dance pop,0.675,0.931,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,70546338.0,176.616,56
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,11862340.0,169.093,56
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,63578139.0,189.052,60


In [54]:
spotify_data.shape

(32824, 20)